<a href="https://colab.research.google.com/github/educatorsRlearners/hugging_face_course/blob/main/03_fine_tuning_a_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3. Fine-Tuning a pretrained model



In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 13.6 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 57.4 MB/s 
     |████████████████████████████████| 596 kB 86.4 MB/s 
     |████████████████████████████████| 895 kB 64.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
